In [1]:
import cv2
import json
import numpy as np

# Cargar la imagen
image = cv2.imread("SoccerNet/calibration/test/00000.jpg")  # Reemplaza con tu ruta
height, width = image.shape[:2]

# Cargar JSON desde archivo
with open('SoccerNet/calibration/test/00000.json', 'r') as f:
    data = json.load(f)

""" # Dibujar líneas
for name, points in data.items():
    color = (0, 255, 0)  # Verde
    thickness = 2

    # Convertir puntos normalizados a pixeles
    pts = [(int(p["x"] * width), int(p["y"] * height)) for p in points]

    if len(pts) == 2:
        cv2.line(image, pts[0], pts[1], color, thickness)
    elif len(pts) > 2:
        cv2.polylines(image, [np.array(pts)], isClosed=False, color=color, thickness=thickness) """

# Mostrar imagen
cv2.imshow("Líneas", image)
cv2.waitKey(0)
cv2.destroyAllWindows()


## Crear Dataset en el formato necesario

In [3]:
import os
import shutil

from_folder = "SoccerNet/calibration"
to_folder = "dataset-det2/images"

# Crear la carpeta destino si no existe
os.makedirs(to_folder, exist_ok=True)

# Iterar sobre cada subcarpeta
dirs = os.listdir(from_folder)
for dir_name in dirs:
    subfolder_path = os.path.join(from_folder, dir_name)
    to_folder_path = os.path.join(to_folder, dir_name)
    if not os.path.isdir(subfolder_path):
        continue  # Saltar si no es carpeta
    
    if not os.path.exists(to_folder_path):
        os.makedirs(to_folder_path)

    # Listar todas las imágenes en la subcarpeta
    images = [img for img in os.listdir(subfolder_path) if img.endswith(".jpg") or img.endswith(".png")]

    # Copiar las imágenes a la carpeta destino
    for img in images:
        src_path = os.path.join(subfolder_path, img)
        dst_path = os.path.join(to_folder_path, img)
        shutil.copy(src_path, dst_path)

print("Imágenes copiadas correctamente.")

Imágenes copiadas correctamente.


In [4]:
import os
import sys
import json
from_folder = "SoccerNet/calibration"
labels_folder = "dataset-det2/labels"

# Diccionario para mapear etiquetas a IDs
class_mapping = {
    "Goal right crossbar": 0,
    "Goal right post left": 0,
    "Goal right post right": 0,
    "Goal left post left": 0,
    "Goal left post left ": 0,
    "Goal left post left": 0,
    "Goal left post right": 0,
    "Goal left crossbar": 0,
    "Small rect. right main": 1,
    "Small rect. right top": 2,
    "Side line right": 3,
    "Side line top": 4,
    "Side line bottom": 5,
    "Big rect. right top": 6,
    "Big rect. right main": 7,
    "Circle right": 8,
    "Side line left": 9,
    "Big rect. left top": 10,
    "Big rect. left bottom": 11,
    "Big rect. left main": 12,
    "Big rect. right bottom": 13,
    "Small rect. right bottom": 14,
    "Small rect. left top": 15,
    "Small rect. left bottom": 16,
    "Small rect. left main": 17,
    "Circle left": 18,
    "Circle central": 19,
    "Middle line": 20,
    "Line unknown": 0,
    "Goal unknown": 0,
}

# Función auxiliar para obtener el bounding box
def get_bbox_from_points(points):
    xs = [p["x"] for p in points]
    ys = [p["y"] for p in points]
    x_min, x_max = min(xs), max(xs)
    y_min, y_max = min(ys), max(ys)
    x_center = (x_min + x_max) / 2
    y_center = (y_min + y_max) / 2
    width = x_max - x_min
    height = y_max - y_min
    #si tiene 2 puntos calculamos el angulo
    pos = 0 #0 horizontal, 1 vertical
    if len(points) == 2:
        x1, y1 = points[0]["x"], points[0]["y"]
        x2, y2 = points[1]["x"], points[1]["y"]
        angle = np.arctan2(y2 - y1, x2 - x1)
        # Convertir a grados
        angle = np.degrees(angle)
        # Normalizar el ángulo entre 0 y 360
        angle = angle % 360
        if angle > 0:
            pos = 1
        else:
            pos = 0
    return x_center, y_center, width, height, pos

# Crear la carpeta destino si no existe
os.makedirs(labels_folder, exist_ok=True)

# Iterar sobre cada subcarpeta
dirs = os.listdir(from_folder)
for dir_name in dirs:
    subfolder_path = os.path.join(from_folder, dir_name)
    to_folder_path = os.path.join(labels_folder, dir_name)
    if not os.path.isdir(subfolder_path):
        continue  # Saltar si no es carpeta
    
    if not os.path.exists(to_folder_path):
        os.makedirs(to_folder_path)

    #listar todos los json
    jsons = [json for json in os.listdir(subfolder_path) if json.endswith(".json")]

    for j in jsons:
        #si el nombre del archivo no es numérico, saltar
        if not j[:-5].isdigit():
            print(f"Nombre de archivo no numérico: {j}")
            continue
        with open(os.path.join(subfolder_path, j), "r") as f:
            try:
                data = json.load(f)
            except Exception as e:
                print(f"Error al cargar el archivo JSON {j}: {e}")
                continue
            label_txt = os.path.join(to_folder_path, j.replace(".json", ".txt"))
            with open(label_txt, "w") as label_file:
                for label, points in data.items():
                    if label not in class_mapping:
                        print(f"Etiqueta desconocida: {label} en {j}")
                    if label not in class_mapping or len(points) < 2:
                        continue
                    class_id = class_mapping[label]
                    x_c, y_c, w, h, pos = get_bbox_from_points(points)
                    label_file.write(f"{class_id + pos*20} {x_c} {y_c} {w} {h}\n")
print("Etiquetas generadas correctamente.")

Nombre de archivo no numérico: match_info.json
Nombre de archivo no numérico: match_info.json
Nombre de archivo no numérico: match_info.json
Etiquetas generadas correctamente.


# Finetunear Modelo

In [6]:
from ultralytics import YOLO

# Cargar un modelo base
model = YOLO('yolo11n.pt')  
# Entrenar (fine-tune) con tus datos
model.train(data='line_dataset_detail2.yaml', epochs=50, imgsz=640, batch=16, workers=16, device="cuda")  # Ajusta los parámetros según tus necesidades

New https://pypi.org/project/ultralytics/8.3.143 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.141 🚀 Python-3.12.3 torch-2.7.0+cu126 CUDA:0 (NVIDIA GeForce RTX 4060, 7911MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=line_dataset_detail2.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train8, nbs=64, nms=False, opset=None, optim

train: Scanning /home/dani/Desktop/futbol/dataset-det2/labels/train.cache... 14513 images, 0 backgrounds, 382 corrupt: 100%|██████████| 14513/14513 [00:00<?, ?it/s]

train: /home/dani/Desktop/futbol/dataset-det2/images/train/00005.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [1.0000093]
train: /home/dani/Desktop/futbol/dataset-det2/images/train/00009.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [1.0000093]
train: /home/dani/Desktop/futbol/dataset-det2/images/train/00019.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [1.0000052]
train: /home/dani/Desktop/futbol/dataset-det2/images/train/00026.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [1.0000093]
train: /home/dani/Desktop/futbol/dataset-det2/images/train/00042.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [1.0000093]
train: /home/dani/Desktop/futbol/dataset-det2/images/train/00332.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [1.0000138]
train: /home/dani/Desktop/futbol/dataset-det2/images/train/00460

val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1157.0±656.6 MB/s, size: 141.3 KB)


val: Scanning /home/dani/Desktop/futbol/dataset-det2/labels/valid.cache... 2796 images, 0 backgrounds, 84 corrupt: 100%|██████████| 2796/2796 [00:00<?, ?it/s]

...

      50/50      2.61G     0.5982      0.696     0.8619         14        640: 100%|██████████| 884/884 [01:46<00:00,  8.33it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 85/85 [00:08<00:00,  9.89it/s]


                   all       2712      26138       0.58      0.527      0.471      0.393

50 epochs completed in 1.612 hours.
Optimizer stripped from /home/dani/Desktop/futbol/runs/detect/train8/weights/last.pt, 5.5MB
Optimizer stripped from /home/dani/Desktop/futbol/runs/detect/train8/weights/best.pt, 5.5MB

Validating /home/dani/Desktop/futbol/runs/detect/train8/weights/best.pt...
Ultralytics 8.3.141 🚀 Python-3.12.3 torch-2.7.0+cu126 CUDA:0 (NVIDIA GeForce RTX 4060, 7911MiB)
YOLO11n summary (fused): 100 layers, 2,590,147 parameters, 0 gradients, 6.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 85/85 [00:10<00:00,  8.05it/s]


                   all       2712      26138       0.58      0.526      0.471      0.393
                Unknow        287        337      0.399       0.12      0.143      0.112
Small rect. right main        126        126      0.251     0.0476      0.123     0.0984
 Small rect. right top         72         72          1          0     0.0548     0.0369
       Side line right        218        218      0.206      0.601      0.253      0.237
         Side line top        231        231      0.272      0.433      0.262      0.185
      Side line bottom         72         72      0.279      0.778      0.387      0.368
   Big rect. right top         91         91          1          0     0.0872     0.0726
  Big rect. right main        250        250       0.27      0.704      0.306       0.28
          Circle right       1038       1038      0.737      0.921      0.917      0.823
        Side line left        281        281      0.248      0.612      0.282      0.263
    Big rect. left to

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x73717a454e90>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,   

In [7]:
#guardar el modelo
model.save("yolo11n_line_detail.pt")